# Adopted from https://gitlab.qdatalabs.com/quantiphi-sagemaker-marketplace-examples/bar-qr-code-detection/blob/master/BarcodeQRcodeScannerNB-ARN.ipynb

## Imports

In [1]:
import sagemaker as sage
import os
import boto3
from sagemaker import get_execution_role
from sagemaker import transformer
from sagemaker import model

role = get_execution_role()

## Create the session

The session remembers our connection parameters to Amazon SageMaker. We'll use it to perform all of our SageMaker operations.

In [2]:
sess = sage.Session()

account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

## Create Model

Now we use the Bar/QR-code Scanner Model Package to create a model

In [3]:
from src.model_package_arns import ModelPackageArnProvider

In [4]:
from sagemaker import ModelPackage
modelpackage_arn = ModelPackageArnProvider.get_model_package_arn(sess.boto_region_name)

# Define predictor wrapper class
def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session, content_type='image/jpeg')

# Create a deployable model
model = ModelPackage(role=role,
                                      model_package_arn = modelpackage_arn,
                                      sagemaker_session = sess,
                                      predictor_cls = predict_wrapper)
# ARN
print(modelpackage_arn)

arn:aws:sagemaker:us-east-1:865070037744:model-package/bar-qr-code-reader-1571413449-a35a47dda86b16474ecd69ff9f20b46f


### Live Inference Endpoint

Now we demonstrate the creation of an endpoint for live inference

In [36]:
# Deploy the model
predictor = model.deploy(1, 'ml.m5.12xlarge', endpoint_name='barcode-endpoint')

Using already existing model: bar-qr-code-reader-1571413449-a35a47dda-2020-04-14-14-36-12-091


-------------!

### Testing on a single file 

The model takes an image file (.jpg/.png) as input. We give the location of the sample input file and prepare it for payload. An example has been shown in the following code shell.

We get the response for our request as a dictionary of Barcode Data and the Barcode Type associated with the barcode. In case of a QR code, the Barcode Type is returned as "QR-code".

In [37]:
file_name = 'data/test_files/SixBoxesInCar2.jpg'

#### Preparing the input file for payload and viewing the response

In [38]:
# Open the file and read the image into a bytearray.
with open(file_name, "rb") as image:
  b = bytearray(image.read())

# Perform a prediction
result = predictor.predict(b).decode('utf-8')

# View the prediction
print(result)

{"N35E-ZCOO-C1VL-9TL5": "QRCODE", "CW7L-R0WW-JF2K-KD2D": "QRCODE", "2888-ABFS-N1IM-D6R0": "QRCODE", "9UIQ-NVR1-DD58-KCYY": "QRCODE", "J2HY-S0O2-G2HV-8T9D": "QRCODE"}


### Clean Up Endpoint

In [13]:
predictor.delete_endpoint()